**Narrowing down DonorsChoose for a quick triage demo** 

In [1]:
import yaml
import pandas as pd
import psycopg2
from triage.experiments import MultiCoreExperiment
from triage import create_engine
from sqlalchemy.engine.url import URL

#### DB Conn

In [2]:
with open('database.yaml', 'r') as f:
    config = yaml.safe_load(f)
    db_url = URL(
        'postgres',
        host=config['host'],
        username=config['user'],
        database=config['db'],
        password=config['password'],
        port=config['port'],
    )

    conn = create_engine(db_url)

Steps:
- Create a sampled down version of the donors choose data
- Create a handful of features to demonstrate the feature engineering capabilities of triage
- run a small model grid with three models (Logit, DT, RF)
- Target is to get the triage run finished in a few mins

#### Creating the Sampled down version of donors

For testing, I'm creating a new projects table that contains prjects from ~10% schools in the dataset and changing the cohort query read from the "new" projects table.

Note -- There are about 57000 different schools. We can change how we sample

In [23]:
q = """

    drop table if exists optimized.projects_sampled_temp; 
    drop table if exists optimized.donations_sampled_temp;
    drop table if exists optimized.precomputed_features_sampled;
    
    -- Sampling schools and projects
    create table optimized.projects_sampled_temp as (
        with schools as (
        select 
            distinct schoolid 
        from optimized.projects
    ),
    sampled_schools as (
        select * from schools order by random() limit 1425
    )
    select 
        *
    from sampled_schools join optimized.projects using(schoolid)
    );
    
    
    -- Fetching all donations from the sampled projects
    create table optimized.donations_sampled_temp as (
        select b.* from optimized.projects_sampled_temp a join optimized.donations b using(entity_id)
    ); 
    
    -- Fetching the precomputed features for the sampled projects
    create table optimized.precomputed_features_sampled as (
        select * from optimized. projects_sampled_temp join optimized.time_series_features using(entity_id, date_posted) 
    );    
"""

conn.execute(q)

#### Triage config

currently contains six features. Two static and two dynamic. But, since the dynamic features are precomputed, from a demo perspective they appear static. We could see how long it takes to compute the features on-the-fly maybe with indexed tables. (currently the optimized donations table doesn't have any indexes)



In [43]:
with open('demo_config.yaml', 'r') as f:
    triage_config = yaml.safe_load(f)
    
# TODO -- replace with an S3 bucket
project_folder = '/mnt/data/experiment_data/donors/demo/'

#### Experiment

In [44]:
experiment = MultiCoreExperiment(
    config=triage_config,
    db_engine=conn,
    n_processes=2,
    n_db_processes=2,
    project_path=project_folder,
    replace=True,
    save_predictions=False
)

2021-09-20 15:44:58 - VERBOSE Matrices and trained models will be saved in /mnt/data/experiment_data/donors/demo/
2021-09-20 15:44:58 -  NOTICE Replace flag is set to true. Matrices, models, evaluations and predictions (if exist) will be replaced
2021-09-20 15:44:58 -  NOTICE Save predictions flag is set to false. Predictions won't be stored in the predictions table. This will decrease both the running time of an experiment and also decrease the space needed in the db
2021-09-20 15:44:58 - VERBOSE Using random seed [1995] for running the experiment
2021-09-20 15:44:58 -  NOTICE bias_audit_config missing in the configuration file or unrecognized. Without protected groups, you will not audit your models for bias and fairness.
2021-09-20 15:44:58 -  NOTICE scoring.subsets missing in the configuration file or unrecognized. No subsets will be generated


In [45]:
%%time
experiment.run()

2021-09-20 15:45:01 - WARNING Section: scoring - No training_metric_groups configured. If training set evaluation metrics are desired, they must be added
2021-09-20 15:45:01 - SUCCESS Experiment validation ran to completion with no errors
2021-09-20 15:45:01 - VERBOSE Computed and stored temporal split definitions
2021-09-20 15:45:01 -    INFO Setting up cohort
2021-09-20 15:45:04 - SUCCESS Cohort setted up in the table cohort_all_entities_1c7e6ce13aaaa12abe16964d559707a9 successfully
2021-09-20 15:45:04 -    INFO Setting up labels
2021-09-20 15:45:19 - SUCCESS Labels setted up in the table labels_quickstart_label_76cc17d016ee3084aed0425d67801eb1 successfully 
2021-09-20 15:45:19 -    INFO Creating features tables (before imputation) 
2021-09-20 15:45:19 -    INFO Creating collate aggregations


/home/kasun/.pyenv/versions/aclu/lib/python3.7/site-packages/sqlalchemy/sql/base.py:302: SAWarning: Can't validate argument 'autoload_from'; can't locate any SQLAlchemy dialect named 'autoload'
  % (k, dialect_name)


2021-09-20 15:45:20 - VERBOSE Starting Feature aggregation
2021-09-20 15:45:20 -  NOTICE Imputed feature table project_features_aggregation_imputed looks good, skipping feature building!
2021-09-20 15:45:21 -  NOTICE Imputed feature table teachr_funding_aggregation_imputed looks good, skipping feature building!
2021-09-20 15:45:21 -  NOTICE Imputed feature table donation_features_aggregation_imputed looks good, skipping feature building!
2021-09-20 15:45:21 -    INFO Processing query tasks with 2 processes
2021-09-20 15:45:21 -    INFO Processing features for project_features_entity_id
2021-09-20 15:45:22 -    INFO Beginning insert batch
2021-09-20 15:45:22 -    INFO Beginning insert batch
2021-09-20 15:45:22 -    INFO Beginning insert batch
2021-09-20 15:45:22 -    INFO Beginning insert batch
2021-09-20 15:45:22 -    INFO Beginning insert batch
2021-09-20 15:45:23 -    INFO Beginning insert batch
2021-09-20 15:45:23 -    INFO Beginning insert batch
2021-09-20 15:45:23 -    INFO Beginn

#### Evaluations

In [46]:
q = "select experiment_hash from triage_metadata.experiment_runs order by start_time desc limit 1;"

experiment_hash = pd.read_sql(q, conn)['experiment_hash'].iloc[0]
experiment_hash

'f48d28dd088aeeb6a87ab9f3f9f95e5a'

In [47]:
q = """
    select 
        to_char(max(train_end_time), 'YYYY-MM-DD') as last_time
    from triage_metadata.experiment_models
    join triage_metadata.models using(model_hash)
    where experiment_hash = '{experiment_hash}'
""".format(experiment_hash=experiment_hash)

last_train_end_time = pd.read_sql(q, conn)['last_time'].iloc[0]
last_train_end_time

'2012-11-01'

In [48]:
q = """
    select 
        model_id, model_type, metric, parameter, best_value, worst_value, stochastic_value
    from triage_metadata.experiment_models
    join triage_metadata.models using(model_hash)
    join test_results.evaluations using(model_id)
    where experiment_hash = '{experiment_hash}' and train_end_time='{last_split}' 
    and metric='precision@' and parameter='15_pct'
""".format(
    experiment_hash = experiment_hash,
    last_split=last_train_end_time
)

evals = pd.read_sql(q, conn)
evals

,model_id,model_type,metric,parameter,best_value,worst_value,stochastic_value
0,53,sklearn.tree.DecisionTreeClassifier,precision@,15_pct,0.524752,0.455446,0.490759
1,54,triage.component.catwalk.estimators.classifier...,precision@,15_pct,0.495050,0.366337,0.436634
2,45,triage.component.catwalk.baselines.rankers.Per...,precision@,15_pct,0.465347,0.465347,0.465347
3,48,sklearn.ensemble.RandomForestClassifier,precision@,15_pct,0.504950,0.504950,0.504950
